In [1]:
import json
import pandas as pd
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
def generate_df(directory):
    json_path = directory + '/_annotations.coco.json'
    with open(json_path,'r') as file:
        data = json.load(file)
    df = []
    for i in range(len(data['annotations'])):
        img = data['images'][i]['file_name']
        if data['annotations'][i]['category_id'] == 1:
            label = '0'
        else:
            label = '1'
        df.append([img, label])
    return pd.DataFrame(df, columns=['img', 'label'])

In [3]:
from google.colab import drive
drive.mount('/content/drive')

path_to_file = '/content/drive/My Drive/brain_tumor'

Mounted at /content/drive


In [4]:
TRAIN_PATH = f'{path_to_file}/train'
TEST_PATH = f'{path_to_file}/test'
VALID_PATH = f'{path_to_file}/valid'

train_df = generate_df(TRAIN_PATH)
test_df = generate_df(TEST_PATH)
valid_df = generate_df(VALID_PATH)

In [5]:
train_df.head()

,img,label
0,2256_jpg.rf.3afd7903eaf3f3c5aa8da4bbb928bc19.jpg,0
1,2871_jpg.rf.3b6eadfbb369abc2b3bcb52b406b74f2.jpg,0
2,2921_jpg.rf.3b952f91f27a6248091e7601c22323ad.jpg,0
3,2777_jpg.rf.3b60ea7f7e70552e70e41528052018bd.jpg,0
4,2860_jpg.rf.3bb87fa4f938af5abfb1e17676ec1dad.jpg,0


In [6]:
test_df.head()

,img,label
0,1706_jpg.rf.004b87f63425ef145855aab826e33eee.jpg,1
1,1363_jpg.rf.01c4537b57f360820b9e00c7e0c99ae1.jpg,1
2,1136_jpg.rf.02df64849485272e48139498c36c8e44.jpg,1
3,2210_jpg.rf.039c38182fe783c69dd28160e5ef02a0.jpg,0
4,2058_jpg.rf.04e8e79db0fbf2459585ce3954f50739.jpg,0


In [7]:
valid_df.head()

,img,label
0,1997_jpg.rf.57e306420710246d04c0d5c26e6496c6.jpg,0
1,2422_jpg.rf.59a35b6d8d2eed150af8535cf09bbde5.jpg,0
2,1630_jpg.rf.5afc6e25d7987503a11bcb0791b60b53.jpg,1
3,1330_jpg.rf.5b23bf18a29b0f50e376b19a5a84f823.jpg,1
4,2768_jpg.rf.5b87c0023909e0789a33185a3aab92e9.jpg,0


In [8]:
datagen = ImageDataGenerator(
    rescale=1./255,
)

In [9]:
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=TRAIN_PATH,
    x_col="img",
    y_col="label",
    target_size=(224, 224),  # Resizing to match VGG16 input
    batch_size=32,
    class_mode='binary'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=valid_df,
    directory=VALID_PATH,
    x_col="img",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 1502 validated image filenames belonging to 2 classes.
Found 429 validated image filenames belonging to 2 classes.


In [10]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Create the model and add custom layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Epoch 1/10
38/46 [=======================>......] - ETA: 3:02 - loss: 0.5539 - accuracy: 0.7615